In [ ]:
##Image to Speech Application

**Image to Audio Story Application** : An application that creates a story of the uploaded photo and makes a voiceover for it.

There will be 3 components of the app:


1.   Image to text model: To cretae a text from the uploaded photo
2.   Text generation model:  to generate a short story
3.   Text to speech model: To generate a audio story



In [1]:
# 1- Image 2 text : The model for the machine to understand what is the scenario based on a photo.



In [ ]:
# 2- Large Language Model : To cretae a short story based on the scenarion from previous model

In [ ]:
# 3- text to speech Model : This model will give voice to the story being created

(113385, 11)
(113385, 11)
(113385, 3)


(26292, 3)